# Multiple Logistic regression

You are working for a new multi-vitamin manufacturing drug company. This company wants to enter the market by producing a new product. They have collected a dataset containing currently available product types in the market and the concentration of various elements associated with them. They have asked you to predict (or classify) the products given these attributes.

The relevant dataset is given in vitamin_product.xlsx. It contains the following columns:

- Na: Concentration of sodium in the product.
- K: Concentration of potassium in the product.
- Ca: Concentration of calcium in the product.
- Fe: Concentration of iron in the product.
- Product: the type of product

Here's a template that assigns several tasks for you and also provides a roadmap to guide you. Please note, you are free to follow your own coding style and name the variables the way you like.

## 1. Load relevant libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## 2. Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Colab Notebooks/CEIC6789_T2_2023/week5/vitamin_product.xlsx'

raw_data = pd.read_excel(path)
raw_data.head()

,Na,K,Ca,Fe,Product
0,13.64,0.06,8.75,0.0,vit_a
1,13.89,0.48,7.83,0.0,vit_a
2,13.53,0.39,7.78,0.0,vit_a
3,13.21,0.57,8.22,0.0,vit_a
4,13.27,0.55,8.07,0.0,vit_a


## 3. Explore descriptive statistics

In [5]:
raw_data.describe(include='all')

,Na,K,Ca,Fe,Product
count,175.000000,175.000000,175.000000,175.000000,175
unique,NaN,NaN,NaN,NaN,3
top,NaN,NaN,NaN,NaN,vit_b
freq,NaN,NaN,NaN,NaN,76
mean,13.384400,0.459143,8.866629,0.059657,NaN
std,0.769317,0.338574,1.385399,0.093310,NaN
min,10.730000,0.000000,5.430000,0.000000,NaN
25%,12.885000,0.145000,8.210000,0.000000,NaN
50%,13.250000,0.560000,8.550000,0.000000,NaN
75%,13.790000,0.610000,9.025000,0.110000,NaN


## 4. Check for multicollinearity

In [6]:
raw_data.corr(method='pearson')

<ipython-input-6-26e863cb5c00>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  raw_data.corr(method='pearson')


,Na,K,Ca,Fe
Na,1.000000,-0.424251,-0.272647,-0.298587
K,-0.424251,1.000000,-0.403987,0.074467
Ca,-0.272647,-0.403987,1.000000,0.165338
Fe,-0.298587,0.074467,0.165338,1.000000


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = raw_data.drop(['Product'], axis=1)

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["features"] = variables.columns

In [8]:
vif

,VIF,features
0,38.071777,Na
1,2.807758,K
2,35.264097,Ca
3,1.514426,Fe


In [10]:
data = raw_data.drop(['Na'], axis=1)

In [11]:
data.corr(method='pearson')

<ipython-input-11-c50c7eb58c83>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr(method='pearson')


,K,Ca,Fe
K,1.000000,-0.403987,0.074467
Ca,-0.403987,1.000000,0.165338
Fe,0.074467,0.165338,1.000000


In [12]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = data.drop(['Product'], axis=1)

vif = pd.DataFrame()
vif["VIF"] = [variance_inflation_factor(variables.values, i) for i in range(variables.shape[1])]
vif["features"] = variables.columns

In [13]:
vif

,VIF,features
0,2.381660,K
1,2.673998,Ca
2,1.457395,Fe


## 5. Logistic regression

### 5.1 Declare inputs and targets

In [14]:
targets = data['Product']
inputs = data.drop(['Product'], axis=1)

### 5.2 Perform feature scaling

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(inputs)

StandardScaler()

In [16]:
inputs_scaled = scaler.transform(inputs)

### 5.3 Divide the dataset into training and testing subsets

In [17]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(inputs_scaled, targets, test_size=0.2, random_state=10)

### 5.4 Build regression model

In [19]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(C=1e9, solver='lbfgs')
log_reg.fit(x_train,y_train)

LogisticRegression(C=1000000000.0)

In [20]:
# Coefficients
log_reg.coef_

array([[ 0.14275602,  0.10608339,  0.19499232],
       [ 0.42875083,  0.44065728,  0.51438334],
       [-0.57150685, -0.54674067, -0.70937567]])

In [21]:
# Intercepts
log_reg.intercept_

array([ 0.45847621,  0.55609367, -1.01456988])

In [22]:
# Classes
log_reg.classes_

array(['vit_a', 'vit_b', 'vit_c'], dtype=object)

- Why are there 3 rows and 3 columns in the coefficient array?
- Why are there 3 intercept values?

Hint: Look at the logit functional form. What does it represent?

### 5.5 Calculate the accuracy of regression on the testing dataset using the confusion matrix

In [24]:
y_test_pred = log_reg.predict(x_test)
y_test_pred

array(['vit_b', 'vit_b', 'vit_b', 'vit_a', 'vit_b', 'vit_a', 'vit_c',
       'vit_a', 'vit_b', 'vit_b', 'vit_b', 'vit_b', 'vit_a', 'vit_a',
       'vit_c', 'vit_a', 'vit_a', 'vit_b', 'vit_b', 'vit_c', 'vit_a',
       'vit_a', 'vit_b', 'vit_a', 'vit_b', 'vit_b', 'vit_b', 'vit_b',
       'vit_b', 'vit_b', 'vit_a', 'vit_b', 'vit_b', 'vit_c', 'vit_a'],
      dtype=object)

In [25]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test,y_test_pred)

print(confusion_matrix)

[[ 4 11  0]
 [ 6  7  0]
 [ 2  1  4]]


In [26]:
accuracy = confusion_matrix.trace() / confusion_matrix.sum() * 100
accuracy

42.857142857142854

## Additional tasks:

- Try removing Ca instead of Na to check if accuracy improves or not.
- Your company has now given you additional features. Use them to see if accuracy increases. For this, use the file "vitamin_product_additional.xlsx"
- Try using lower values of C in the LogisticRegression class. This will reduce overfitting.

# Use Decision Tree classifier to do the same task

### Build the decision tree model

In [27]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(random_state=42,max_depth=3, min_samples_leaf=5)

dtree.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=3, min_samples_leaf=5, random_state=42)

### Check accuracy

In [28]:
y_test_pred = dtree.predict(x_test)
y_test_pred

array(['vit_a', 'vit_a', 'vit_a', 'vit_a', 'vit_a', 'vit_b', 'vit_c',
       'vit_a', 'vit_b', 'vit_b', 'vit_a', 'vit_b', 'vit_c', 'vit_a',
       'vit_c', 'vit_b', 'vit_b', 'vit_b', 'vit_b', 'vit_c', 'vit_b',
       'vit_b', 'vit_b', 'vit_b', 'vit_a', 'vit_b', 'vit_a', 'vit_a',
       'vit_b', 'vit_a', 'vit_a', 'vit_b', 'vit_a', 'vit_c', 'vit_c'],
      dtype=object)

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix = confusion_matrix(y_test,y_test_pred)

print(confusion_matrix)

[[10  5  0]
 [ 4  9  0]
 [ 0  1  6]]


In [30]:
accuracy = confusion_matrix.trace() / confusion_matrix.sum() * 100
accuracy

71.42857142857143